<a href="https://colab.research.google.com/github/pappuyadav/Squat-Detection-Using-VGG16-and-LSTM/blob/master/squat_detect_for_test_data_keypoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.14.0

In [0]:
#Before testing with any new video dataset please first run this section of code and then upload keypoint files to the folder "/content/testkeypoints"
#Make sure to also load trained model and weights files ("trained_data_squats_keypoints.h5" and "trained_data_squats_keypoints.json")
!mkdir testkeypoints


In [0]:
#Use this for Test Dataset
# Please use this section to test the model using your own test data sets keypoints generated from video image frames
# load json and create model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import sequence
import os
import numpy as np
from keras.utils import np_utils
from sklearn.utils import shuffle
from keras.models import model_from_json
import os
import tensorflow as tf
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

json_file = open('/content/trained_data_squats_keypoints.json', 'r')
loaded_model_json=json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/content/trained_data_squats_keypoints.h5")
print("The model weights have been loaded")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
loaded_model.summary()
#Loading Keypoints from Test Data Set 

#Defining a function to read json files i.e. keypoints files
def read_josn(filepath):
  with open(filepath,'r')as f:
    data=json.load(f)
  return(data)

#Loading Test Data Set Keypoints
test_keypoints=[]
for file in os.listdir('/content/testkeypoints'):
  full_path='/content/testkeypoints/'+ str(file)
  #!rmdir -rf /content/nosquattestkeypoints/.ipynb_checkpoints
  aa=read_josn(full_path)
  df=pd.DataFrame(aa)
  df=df['people'][0]['pose_keypoints_2d']
  test_keypoints.append(df)

#Creating test array
test_keypoints=np.array(test_keypoints)
test_keypoints=test_keypoints.reshape(len(test_keypoints),1,75)
timestep=1
test_data = sequence.pad_sequences(test_keypoints, maxlen=timestep) 
#Now the test dataset is prepared for making predictions
#Using the pretrained model to make predictions of test data keypoints
temp=loaded_model.predict(test_data)
temp1=loaded_model.predict_classes(test_data)
yhat_probs = temp[:,0]
yhat_probs=yhat_probs[:,0]
b=np.array(yhat_probs)
yhat_classes = temp1[:,0]

video_length=4     # Please enter length of video file in seconds
fps=len(test_keypoints)/video_length              
disp_time=1/fps
time=[]

#Plotting the time-label grapgh
#Please update plot title and plot figure save file name accordingly
for i in range(len(b)):
  step=disp_time*i
  time.append(step)

fig, ax = plt.subplots()
ax.plot(time,b)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 0.25))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
plt.xlabel("Time(seconds)")
plt.ylabel("Squat-Pose Label")
plt.title("Time-Label Plot for 6600_2")
plt.grid(True)
plt.savefig('plot_6600_2.png')


groupdata=[]
for i in range(len(b)):
  #print(i)
  c=[time[i],b[i]]
  groupdata.append(c)


#Writing the output JSON file for "squats" that contain pairs of data.
#The first value of each pair represents time and the second value represents probability of the squat label
data={}
data['squats']=[]
data['squats'].append(groupdata)
f = open("data_6600_2.json","w")   #Please update file name accordingly
f.write(str(data))
f.close()
